In [ ]:
import os
import sqlite3
import pandas as pd
import requests
import plotly.express as px

## binance data


In [ ]:
# binance constants
BINANCE_API_URL = "https://fapi.binance.com"
BINANCE_ENDPOINT = "/fapi/v1/klines"
BINANCE_ASSETS = ["1000BONKUSDT", "LINKUSDT", "AVAXUSDT", "INJUSDT", "1000PEPEUSDT"]
BINANCE_START_TIMESTAMP = 1704085200000
BINANCE_END_TIMESTAMP = 1706677200000

df_binance_prices = pd.DataFrame()
for asset in BINANCE_ASSETS:
    last_timestamp = BINANCE_START_TIMESTAMP
    while last_timestamp < BINANCE_END_TIMESTAMP:
        params = {
            "symbol": asset,
            "interval": "5m",
            "limit": 1000,
            "startTime": last_timestamp,
        }
        response = requests.get(f"{BINANCE_API_URL}{BINANCE_ENDPOINT}", params=params)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(
                data,
                columns=[
                    "open_time",
                    "open",
                    "high",
                    "low",
                    "close",
                    "volume",
                    "close_time",
                    "quote_asset_volume",
                    "number_of_trades",
                    "taker_buy_base_asset_volume",
                    "taker_buy_quote_asset_volume",
                    "ignore",
                ],
            )
            print(
                f"Successfully fetched {asset} data from {last_timestamp} to {df['close_time'].max()}"
            )
            df["asset"] = asset
            df_binance_prices = pd.concat([df_binance_prices, df])
            last_timestamp = df["close_time"].max()

        else:
            print(f"Error fetching data: {response.status_code}")
            print(f"Error: {response.text}")

In [ ]:
for asset in BINANCE_ASSETS:
    start_time = df_binance_prices[df_binance_prices["asset"] == asset][
        "open_time"
    ].min()
    end_time = df_binance_prices[df_binance_prices["asset"] == asset]["open_time"].max()

    # convert to readable dates
    print(
        f"{asset}: {pd.to_datetime(start_time, unit='ms')} - {pd.to_datetime(end_time, unit='ms')}"
    )

In [ ]:
df_binance_prices.to_csv("../../data/ml_export_binance_prices.csv", index=False)